In [1]:
%load_ext autoreload
%autoreload 2 
%matplotlib tk

import os 
import time 
import pianoq
import datetime
from TC300_COMMAND_LIB import TC300
from pianoq.lab.asi_cam import ASICam
from pianoq.lab.photon_counter import PhotonCounter
from pianoq.lab.thorlabs_motor import ThorlabsKcubeDC, ThorlabsKcubeStepper
from pianoq.lab.photon_scan import PhotonScanner

can't import TimeTagger


In [ ]:
tc = TC300()
serial = tc.list_devices()[0][0]
tc.open(serial, 115200, 3)
tc.set_target_temperature(2, 75)

In [ ]:
tc.close()

In [ ]:
cam = ASICam()
base_path = r'G:\My Drive\People\Ronen\PHD\SPDC2025\phase_matching'
cam.set_exposure(1)

In [ ]:
cam.close()

## Find phase matching temperature

In [ ]:
t = 0
while t < 72:
    time.sleep(2)
    tt = [0]
    tc.get_actual_temperature(2, tt)
    t = round(tt[0], 4)
    cam.save_image(os.path.join(base_path, f"T={t}.fits"))

## Initial single counts after MMF

In [3]:
cam = ASICam()

In [ ]:
cam.set_gain(0)
cam.set_binning(1)
cam.set_exposure(5)
cam.set_roi(2500, 1400, 600, 600)

In [12]:
cam.show_image()
cam.save_image(r'G:\My Drive\People\Ronen\PHD\SPDC2025\After MMF\no_pump_lens')

## Check photon counter 

In [21]:
ph = PhotonCounter(serial_port='COM8')

In [23]:
ph.close()

In [22]:
ph.read()

(array([0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0.]),
 1.0)

In [ ]:
while True:
    x = ph.read()
    print(x[0])

In [16]:
ph.close()

## Photon scan 

In [2]:
timeout = 50000
x_motor = ThorlabsKcubeDC(serial_number=27253522, timeout=timeout)
y_motor = ThorlabsKcubeStepper(serial_number=26001271, timeout=timeout)
ph = PhotonCounter(serial_port='COM8')

In [3]:
x_motor.move_relative(0.1)
y_motor.move_relative(-0.1)

In [4]:
x_motor.timeout = 50000
y_motor.timeout = 50000

In [3]:
print(x_motor.get_position())
print(y_motor.get_position())

15.0
14.7500047


In [5]:
ph.close()

In [6]:
ph = PhotonCounter(serial_port='COM8', integration_time=5)

In [7]:
integration_time = 5 
start_x = 15 # 16.5
start_y = 13.25 # 14.75  
x_pixels = 30
y_pixels = 30
pixel_size_x = 0.05
pixel_size_y = 0.05
# name = 'find_image'
name = 'dx=0.05_no_pump_lens_singles_50K_and_diffuser_0.25deg_integration_5s'

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
dir_path = r'G:\My Drive\People\Ronen\PHD\SPDC2025\phase_matching\Scans'

path = f'{dir_path}\\{timestamp}_{name}.scan'
scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y,
                        run_name=name, is_timetagger=False, coin_window=2e-9, saveto_path=path)

In [8]:
single1s, single2s, coincidences = scanner.scan(x_motor=x_motor, y_motor=y_motor, ph=ph)

Moving to starting position...
starting scan
dur: 0. pix: 0, 0. Singles1: 45343. Singles2: 10631. Coincidence: 4.20.
dur: 6. pix: 0, 1. Singles1: 45112. Singles2: 10736. Coincidence: 3.00.
dur: 12. pix: 0, 2. Singles1: 57579. Singles2: 24016. Coincidence: 14.00.
dur: 17. pix: 0, 3. Singles1: 48158. Singles2: 17060. Coincidence: 5.80.
dur: 23. pix: 0, 4. Singles1: 45413. Singles2: 15748. Coincidence: 4.00.
dur: 29. pix: 0, 5. Singles1: 45321. Singles2: 18052. Coincidence: 6.60.
dur: 35. pix: 0, 6. Singles1: 45318. Singles2: 17655. Coincidence: 6.40.
dur: 41. pix: 0, 7. Singles1: 45317. Singles2: 17894. Coincidence: 7.00.
dur: 46. pix: 0, 8. Singles1: 45331. Singles2: 19041. Coincidence: 7.00.
dur: 52. pix: 0, 9. Singles1: 45401. Singles2: 19719. Coincidence: 8.80.
dur: 58. pix: 0, 10. Singles1: 45504. Singles2: 19662. Coincidence: 6.80.
dur: 64. pix: 0, 11. Singles1: 45345. Singles2: 20029. Coincidence: 8.40.
dur: 70. pix: 0, 12. Singles1: 45406. Singles2: 21530. Coincidence: 7.60.
dur:

In [17]:
# x_motor.close() # pesky bug?
y_motor.close()  
ph.close()

In [9]:
ph.close()

In [12]:
ph = PhotonCounter(serial_port='COM8')

In [7]:
while True:
    print(ph.read_interesting())

(np.float64(48586.0), np.float64(5408.0), np.float64(1.0))
(np.float64(48897.0), np.float64(5561.0), np.float64(2.0))
(np.float64(48516.0), np.float64(5479.0), np.float64(4.0))
(np.float64(49074.0), np.float64(5519.0), np.float64(2.0))
(np.float64(48944.0), np.float64(5510.0), np.float64(0.0))
(np.float64(48593.0), np.float64(5539.0), np.float64(2.0))
(np.float64(49030.0), np.float64(5583.0), np.float64(2.0))
(np.float64(51901.0), np.float64(7849.0), np.float64(2.0))
(np.float64(62697.0), np.float64(16677.0), np.float64(12.0))
(np.float64(62857.0), np.float64(16828.0), np.float64(11.0))
(np.float64(60716.0), np.float64(14839.0), np.float64(10.0))
(np.float64(65159.0), np.float64(18486.0), np.float64(9.0))
(np.float64(112131.0), np.float64(64140.0), np.float64(59.0))
(np.float64(93594.0), np.float64(47494.0), np.float64(40.0))
(np.float64(88405.0), np.float64(40967.0), np.float64(24.0))
(np.float64(89897.0), np.float64(42728.0), np.float64(31.0))
(np.float64(90352.0), np.float64(44818.0

KeyboardInterrupt: 